In [ ]:
import pandas as pd

import folium
from folium.plugins import MarkerCluster

In [ ]:
def data_import_bases(stats = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]):

    """
    Returns a cleaned dataframe with all the military installations.

            Parameters:
                        states (list): list of stats you want to see bases off

            Data:
                        https://public.opendatasoft.com/explore/dataset/military-bases/export/?dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6Im1pbGl0YXJ5LWJhc2VzIiwib3B0aW9ucyI6e319LCJjaGFydHMiOlt7ImFsaWduTW9udGgiOnRydWUsInR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQVZHIiwieUF4aXMiOiJvYmplY3RpZF8xIiwic2NpZW50aWZpY0Rpc3BsYXkiOnRydWUsImNvbG9yIjoiI0ZGNTE1QSJ9XSwieEF4aXMiOiJjb21wb25lbnQiLCJtYXhwb2ludHMiOjUwLCJzb3J0IjoiIn1dLCJ0aW1lc2NhbGUiOiIiLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlfQ%3D%3D&location=6,37.78808,-111.09375&basemap=jawg.light

            Returns:
                        bases_df (DataFrame): 
    """

    # Choose columns we need
    data_columns = ['Geo Point', 'COMPONENT', 'Site Name', 'State Terr', 'Oper Stat']

    # read in alle the military bases - seperator ';'
    bases_df = pd.read_csv('./data/military-bases.csv', sep=';', usecols = data_columns )

    # rename columns for easier use
    bases_df.rename(columns={'Geo Point': 'geoPoint', 'COMPONENT': "component", 'Site Name': 'name', 'State Terr': 'state', 'Oper Stat': 'status'}, inplace = True)

    # checking it there is only Active and Inaktive
    #bases_df[(bases_df.status != 'Active') & (bases_df.status != 'Inactive')]

    # drop all Inaktive bases
    bases_df = bases_df[bases_df.status != 'Inactive']

    # convert geo points
    bases_df[['lat', 'lon']] = bases_df.geoPoint.str.split(',', expand = True).astype('float64')
    bases_df.geoPoint = list(zip(bases_df.lat, bases_df.lon))
    bases_df.drop(columns = ['lat', 'lon'], inplace =  True)

    # filter by list

    bases_df = bases_df[bases_df.state.isin(stats)]

    return bases_df

In [ ]:
west_coast = ["Alaska", "California", "Guam", "Hawaii", "Oregon", "Washington"]

data_import_bases(west_coast)

In [ ]:
west_coast = ["Alaska", "California", "Guam", "Hawaii", "Oregon", "Washington"]
data_df = data_import_bases(west_coast)

# folium Map centered on Glasgow Hall
nps_lat = 36.598802
nps_lon = -121.877178

# Create a map object
# Specify center location, and starting zoom level (0 to 18)
map = folium.Map(location=[nps_lat, nps_lon], zoom_start = 4, control_scal = True, tiles = "Cartodb Positron")

coord_list = data_df.geoPoint

popups = ['<b>Base:</b><br>{}<br><b>Altitude:</b><br>{}'.format(name, 'Null') for (name) in data_df.name.values]

marker_cluster = MarkerCluster(
    locations = coord_list,
    popups = popups,
    name='US Bases & Tide-sensors',
    overlay=True,
    control=True
)

marker_cluster.add_to(map)

folium.LayerControl().add_to(map)
    
map

In [ ]:
import gzip
import os
import pandas as pd

In [ ]:
for file in os.listdir('./data/tide_sensors/.'):

    sensors = dict()

    with gzip.open('./data/tide_sensors/' + file, 'rt') as file_in:
        
        # Reading in header
        sensor_information_file = file_in.readlines()[0:10]

        sensor_information_dict = dict()

        for idx, line in enumerate(sensor_information_file[:-3]):
            
            key, value = line.removeprefix('// ').split(':')
            key = key.strip()
            value = value.strip()

            sensor_information_dict[key] = value

        print(sensor_information_dict)

        # Reading in data
        csv_header = sensor_information_file[-1].removeprefix('// ').split(',')

    sensor_data = pd.read_csv('./data/tide_sensors/' + file, skiprows=10, sep='\t', header=None)
    sensor_data.columns = csv_header

    sensors[sensor_information_dict.get('NOS ID')] = {'Info' : sensor_information_dict, 'Data' : sensor_data}

    print(sensors)

    break
  

In [ ]:
data = pd.read_csv('./data/tide_sensors/' + file, skiprows=10, sep='\t', header=None)
data.columns = csv_header
data.head()

In [133]:
sensor_information_dict = dict()

for file in os.listdir('./data/tide_sensors/.'):

    with gzip.open('./data/tide_sensors/' + file, 'rt') as file_in:
        
        # Reading in header
        sensor_information_file = file_in.readlines()[0:10]

    for line in sensor_information_file[:-4]:
        
        name, value = line.removeprefix('// ').strip().split(':')

        if name in sensor_information_dict:
            sensor_information_dict[name].append(value)
        else :
            sensor_information_dict[name] = [value]

sensor_information_df = pd.DataFrame(sensor_information_dict)

sensor_information_df = sensor_information_df.assign(geoPoint = list(zip(sensor_information_df.Latitude.astype('float64'), sensor_information_df.Longitude.astype('float64'))))
sensor_information_df.drop(columns = ['Latitude', 'Longitude'], inplace =  True)

sensor_information_df.info()

    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NOS ID            5 non-null      object
 1   Location Name     5 non-null      object
 2   Horizontal Datum  5 non-null      object
 3   Operator          5 non-null      object
 4   geoPoint          5 non-null      object
dtypes: object(5)
memory usage: 332.0+ bytes


In [134]:
sensor_information_df.head()

,NOS ID,Location Name,Horizontal Datum,Operator,geoPoint
0,9410170,"SAN DIEGO, SAN DIEGO BAY",WGS-84,DOC>NOAA>NOS>CO-OPS,"(32.71419, -117.17358)"
1,9410660,"LOS ANGELES, OUTER HARBOR",WGS-84,DOC>NOAA>NOS>CO-OPS,"(33.72, -118.272)"
2,9414523,"REDWOOD CITY, WHARF 5, S. F. BAY",WGS-84,DOC>NOAA>NOS>CO-OPS,"(37.50669, -122.20999)"
3,9446484,"TACOMA, COMMENCEMENT BAY",WGS-84,DOC>NOAA>NOS>CO-OPS,"(47.26667, -122.41333)"
4,9447130,"SEATTLE, PUGET SOUND",WGS-84,DOC>NOAA>NOS>CO-OPS,"(47.60263, -122.3393)"
